In [1]:
import numpy as np
import pandas
import string
import urllib
from bs4 import BeautifulSoup as bs
import datetime
import dateutil
from pprint import pprint
import os
import re
print('ok')

ok


In [4]:
# Read dataset
os.chdir('/home/idies/workspace/raddick_folksonomy/data/')
os.getcwd()
inmusic_df = pandas.read_csv('cleaned_audio_data.csv')
# Use music_df for processing; move rows to output when scored
music_df = inmusic_df
# IDs are unique, we can use them as pandices IWW - but now let's use rownumber #music_df.groupby('ID').size().sort_values(ascending=False)
print('Datafile read')

# rename columns
music_df.index.name = 'rownumber'
music_df = music_df.rename(columns={'ID': 'id', 'Source': 'source', 'Sample File Name': 'filename'})
print('Columns renamed')

### SUMMARY STATISTICS FOR FILENAME DATASET
print(len(music_df))
# 2,973 filenames

print('\n')
print(
    music_df
    .groupby('source')
    .size()
    .sort_values(ascending=False)
)
# Sources:
#DimeaDozen      1804
#Etree            564
#LosslessLegs     275
#TradersDen       330

print('\n')
print(
    music_df['filename'].
    groupby(music_df['filename'].
            apply(lambda x: x.split('.')[-1])).
    size().
    sort_values(ascending=False)
)
# File extensions:
#flac    2798
#shn       92
#mp2       48
#mp3       31
#wav        1
#cue        1
#aud        1
#FLAC       1
print('ok')

Datafile read
Columns renamed
2973


source
DimeaDozen      1804
Etree            564
TradersDen       330
LosslessLegs     275
dtype: int64


filename
flac    2798
shn       92
mp2       48
mp3       31
wav        1
cue        1
aud        1
FLAC       1
Name: filename, dtype: int64
ok


In [ ]:
'''
STEPS IN FLOWCHART:
A. Does filename start with the artist name? (False -> 0)
B. Does the date come next? (False -> 0)
C. Does an artist name come next? (False -> 0)

D. Does the artist have an official abbrev? (True -> E, False -> G)

E. Is (abbrev used correctly) OR (date correctly formatted)? (True -> F , False -> 1)
F. Is (abbrev used correctly) AND (date correctly formatted)? (True -> H , False -> 2)

G. Is the date correctly formatted? (True -> H, False -> 1)

H. Does the date immediately follow artist name, no space between? (False -> 3)

J. Do they include the mics parameter? (True -> K, False -> L)
K. Do they use the mics parameter correctly? (True -> L, False -> 4)

L. Is taper used correctly? (False -> 4)
M. Is sbesource used correctly? (False -> 4)
N. Is filetype used correctly? (False -> 4)
P. Does filetype accurately describe the type of file? (True -> 5, False -> 4)
'''
print('ok')

In [5]:
# GET LIST OF OFFICIAL ABBREVIATIONS

pageurl = 'http://wiki.etree.org/index.php?page=BandAbbreviations'
page = urllib.request.urlopen(pageurl).read()
soup = bs(page, "html.parser")
tables = soup.find_all('table')

#for thistable in tables:
#    print(thistable)
#    print('---------')

bandtable = tables[1]
bands = bandtable.find_all('tr')
allbands = []

print('Thinking...')
for thisband in bands[1:]:
    theirdata = thisband.find_all('td')
    oneband = {}
    oneband['name'] = theirdata[0].text
    oneband['abbrev'] = theirdata[1].text
    try:
        oneband['otherabbrev'] = theirdata[2].text
    except:
        pass
    try:
        oneband['notes'] = theirdata[3].text
    except:
        pass
    allbands.append(oneband)
abbrev_df = pandas.DataFrame.from_records(allbands)
abbrev_df.index.name = 'rownumber'

column_order = ['name', 'abbrev', 'otherabbrev', 'notes']
abbrev_df = abbrev_df[column_order]
abbrev_df = abbrev_df.fillna('')

# Trim one space from beginning and end of band name
abbrev_df['name'] = abbrev_df['name'].apply(lambda x: str(x).strip())

# Trim away brackets from beginning and end of band name (for some bands)
abbrev_df['name'] = abbrev_df['name'] = np.where((abbrev_df['name'].apply(lambda x: (str(x)[0])) == '['), 
                             abbrev_df['name'].apply(lambda x: str(x)[1:]), 
                             abbrev_df['name']) 

abbrev_df['name'] = abbrev_df['name'] = np.where((abbrev_df['name'].apply(lambda x: (str(x)[len(str(x))-1])) == ']'),
                              abbrev_df['name'].apply(lambda x: str(x)[:-1]), 
                              abbrev_df['name'])

abbrev_df['abbrev'] = abbrev_df['abbrev'].apply(lambda x: str(x).strip())
#abbrev_df['abbrev'].apply(lambda x: '|'+str(x)+'|')

abbrev_df.to_csv('abbreviations.csv')
os.listdir()
#print('ok')
pprint(abbrev_df.values.tolist())

Thinking...
[['3 Apples High', '3ah', '', ''],
 ['A Band of Gypsys', 'bog', 'jhbg ', ''],
 ['Alan Hertz and Friends', 'ahf', ' ahp,ah&f ', ' AbbrevDiscussion '],
 ['Allman Brothers Band', 'abb', '', ''],
 ['Amfibian', 'amf', '', ''],
 ['Barbara Cue', 'bcue', '', ''],
 ['Barefoot Manner', 'bfm', '', ''],
 ['Beanlan]', 'beanland', '', ''],
 ['Bela Fleck & the Flecktones', 'bfft', '', ''],
 ['Bela Fleck & Tony Trischka', 'bftt', '', ''],
 ['Ben Harper', 'bh', '', ''],
 ['Ben Harper and the Innocent Criminals', 'bhic', ' ', ''],
 ['Big In Japan', 'bij', '', ''],
 ['Big Smith', 'bs', '', ''],
 ['The Big Wu', 'wu', '', ''],
 ['Black Crowes', 'bc', ' crowes ', ''],
 ['Blind Melon', 'bmelon', '', ''],
 ['Blueground Undergrass', 'bgug', '', ''],
 ['Blues Traveler', 'bt', ' ', '   Btat:Blues+Traveler '],
 ["Bockman's Euphio", 'be', '', ''],
 ['Brothers Past', 'bp', '', ''],
 ['Bruce Hornsby', 'bruce', ' bh ', ' [AbbrevDiscussion] '],
 ['El Buho', '', ' buho ', ''],
 ['Built to Spill', 'bts', '',

In [9]:
os.chdir('/home/idies/workspace/raddick_folksonomy/data/')
os.getcwd()
inmusic_df = pandas.read_csv('cleaned_audio_data.csv')
# Use music_df for processing; move rows to output when scored
music_df = inmusic_df
# IDs are unique, we can use them as pandices IWW - but now let's use rownumber #music_df.groupby('ID').size().sort_values(ascending=False)

# rename columns
music_df.index.name = 'rownumber'
music_df = music_df.rename(columns={'ID': 'id', 'Source': 'source', 'Sample File Name': 'filename'})
 
# Initialize score to -9999
music_df['score'] = -9999

# Create outmusic_df as the place to move data to when a score has been assigned
thecolumns = music_df.columns
outmusic_df = pandas.DataFrame(data=None, columns=thecolumns) # Trick to copy schema but not data
outmusic_df.index.name = music_df.index.name

# All filenames have a period somewhere
#music_df[music_df['filename'].apply(lambda x: '.' not in x)]

# First, get filetype (whatever is after the final period)
music_df['filetype'] = music_df['filename'].apply(lambda x: x.split('.')[-1].strip())
music_df['remainder'] = music_df['filename'].apply(lambda x: x[0:x.rfind('.')])

# If there are no numbers at all, there's no date, so score as zero
music_df.loc[music_df['remainder'].apply(lambda x: re.search('\d', x) == None), 'score'] = 0
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score']][music_df['remainder'].apply(lambda x: re.search('\d', x) == None)])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]



# Look for things that meet the standard fully: bbyyyy-mm-dd
# first, parse to first number - that's where the date starts (usually, but not for U2 or 4 Non-Blondes)

music_df['band_maybe'] = music_df['remainder'].apply(lambda x: x[0:re.search('\d', x).start()])
#print(len(outmusic_df))
#print(len(music_df))
#outmusic_df
music_df['abbrev_ok'] = False
music_df.loc[music_df['band_maybe'].isin(abbrev_df['abbrev'][abbrev_df['abbrev'] != '']), 'abbrev_ok'] = True
# abbrev_ok = 1: 296

# If abbrev_ok, cut band name off front of remainder
music_df.loc[music_df['abbrev_ok'], 'remainder'] = music_df['remainder'][music_df['abbrev_ok']].apply(lambda x: x[re.search('\d', x).start():])
music_df['thedate'] = pandas.to_datetime(music_df['remainder'][music_df['abbrev_ok']].apply(lambda x: x[0:10]), errors='coerce')

music_df['mics_etc'] = music_df['remainder'][music_df['thedate'].notnull()].apply(lambda x: x[11:])
music_df.loc[music_df['mics_etc'].notnull(), 'score'] = 4

outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 4]).sort_values(by='score', ascending=False)
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]

neworder = ['score', 'band_maybe', 'thedate', 'mics_etc', 'filetype', 'source', 'filename', 'id']
outmusic_df = outmusic_df[neworder]

print('Filenames processed: {0:,.0f}'.format(len(outmusic_df)))
print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

#outmusic_df.to_csv('outmusic.csv')
#print('File written')
print('\n')
print('Three rows from the output data, selected at random:')
outmusic_df.sample(3)

Filenames processed: 206
Filenames remaining: 2,767


Three rows from the output data, selected at random:


,score,band_maybe,thedate,mics_etc,filetype,source,filename,id
rownumber,,,,,,,,
2767,4,pj,1998-09-03,2t01,flac,TradersDen,pj1998-09-03d2t01.flac,126947.html
306,4,wsp,2016-06-30,1t12,flac,Etree,wsp2016-06-30s1t12.flac,587364.html
348,4,wsp,2016-07-03,2t01,flac,Etree,wsp2016-07-03s2t01.flac,587410.html
